In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
import numpy as np 

# Spectral Clustering Methods

<h5>1. Newman Leading Eigenvector Clustering</h5>

In [ ]:
def newman_leading_eigenvector( G ) :
    A_not_sparse = []
    for _ , nbrdict in G.adjacency():
        A_not_sparse.append( list( nbrdict.keys() ) )

    NODES = len(A_not_sparse)
    A = np.zeros( ( NODES , NODES ) )

    for i , adjacency_list in enumerate( A_not_sparse ):
        for adj_node in adjacency_list :
            A[ i ][ int(adj_node) ] = 1

    D_vect = np.matmul( A , np.ones( ( A.shape[0] , 1 ) ) )
#     B = mdoularity_matrix
    B = A - np.matmul( D_vect , D_vect.T ) / np.sum( A )
    eig_values , eig_vectors = np.linalg.eig( B )
    copy_eigen_values = list(eig_values)
    copy_eigen_values.sort( )
    SL_eigen_value = copy_eigen_values[-1]
    index_sl_largest_eigenvalue , = np.where( eig_values == SL_eigen_value )
    eigen_vector = eig_vectors[ : , index_sl_largest_eigenvalue ]
    eigen_vector.sort()
#     plt.plot( range(NODES) , eigen_vector )
#     plt.show(  )
    comm1 , comm2 = show_community( eig_vectors[ : , index_sl_largest_eigenvalue ] ) 
    print( comm1 , comm2 )
    return [ comm1 , comm2 ]
 

<h5>2. Newman Second Largest EigenVector Clustering</h5>

In [ ]:
def newman_sl_eigenvector_community( G ):
    A_not_sparse = []
    for _ , nbrdict in G.adjacency():
        A_not_sparse.append( list( nbrdict.keys() ) )

    NODES = len(A_not_sparse)
    A = np.zeros( ( NODES , NODES ) )

    for i , adjacency_list in enumerate( A_not_sparse ):
        for adj_node in adjacency_list :
            A[ i ][ int(adj_node) ] = 1

    D = np.power( np.matmul( A , np.ones( ( A.shape[0] , ) ) ) , -0.5 )
    D_1_2 = np.diag( D )
    L = np.matmul( np.matmul( D_1_2 , A ) , D_1_2)
    
    eig_values , eig_vectors = np.linalg.eig( L )
    copy_eigen_values = list(eig_values)
    copy_eigen_values.sort( )
    SL_eigen_value = copy_eigen_values[-2]
    index_sl_largest_eigenvalue , = np.where( eig_values == SL_eigen_value )
    eigen_vector = eig_vectors[ : , index_sl_largest_eigenvalue ]
    eigen_vector.sort()
#     plt.plot( range(NODES) , eigen_vector )
#     plt.show(  )
    print( show_community( eig_vectors[ : , index_sl_largest_eigenvalue ] ) )
 

<h5>3. Fiedler Vector Clustering</h5>

In [ ]:
def fiedler_vector_method( G ):
    A_not_sparse = []
    for _ , nbrdict in G.adjacency():
        A_not_sparse.append( list( nbrdict.keys() ) )

    NODES = len(A_not_sparse)
    A = np.zeros( ( NODES , NODES ) )

    for i , adjacency_list in enumerate( A_not_sparse ):
        for adj_node in adjacency_list :
            A[ i ][ int(adj_node) ] = 1
    D = np.diag( np.matmul( A , np.ones( ( A.shape[0] ,  ) ) ) )
    L = D - A
    eig_values , eig_vectors = np.linalg.eig( L )
    copy_eigen_values = list(eig_values)
    copy_eigen_values.sort( )
    SS_eigen_value = copy_eigen_values[1]
    index_ss_eigenvalue , = np.where( eig_values == SS_eigen_value )
    eigen_vector = eig_vectors[ : , index_ss_eigenvalue ]
    eigen_vector.sort()
#     plt.plot( range(NODES) , eigen_vector )
#     plt.show(  )
    comm1 , comm2 = show_community( eig_vectors[ : , index_ss_eigenvalue ] ) 
    print( comm1 , comm2 )
    return [ comm1 , comm2 ]
    

# Non-Spectral Methods

<h5>1. Girvan-Newman</h5>

In [ ]:
def girvan_newman( G ):
    communities_generator = community.girvan_newman(G)
    top_level_communities = next(communities_generator)
    next_level_communities = next(communities_generator)
    return sorted(map(sorted, top_level_communities))

In [ ]:
def show_community( eigenvector ):
    comm1 = []
    comm2 = []
    for i , element in enumerate(eigenvector):
        if element < 0:
            comm1.append( i )
        else:
            comm2.append( i )
    return comm1 , comm2

<h5>2. Label Propagation Method</h5>

In [ ]:
def label_propagation_method( G ):
    communities_generator = community.label_propagation.asyn_lpa_communities(G)
    comm_list = []
    for comm in communities_generator:
        comm = list(comm)
        comm.sort()
        comm_list.append(comm )
    print(comm_list)

In [ ]:
def generate_community_labels( comm1 , comm2 ):
    labels = np.ones( len(comm1) + len(comm2) , )
    for node in comm1:
        labels[ node ] = 0
    for node in comm2:
        labels[ node ] = 1
    return labels
    

# DataSets

<h5>1. Zachary's Karate Club</h5>

In [ ]:
ground_truth_zach = [ 0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1 ]
len(ground_truth_zach)
G = nx.karate_club_graph()
G.degree()

In [ ]:
newman_leading_eigenvector(G) # 2 is incorrectly labelled
newman_sl_eigenvector_community( G )
girvan_newman( G )
fiedler_vector_method( G )

<h5>2. Facebook</h5>

In [ ]:
NODES = 4039
G = nx.read_edgelist("datasets/facebook_combined.txt.gz")

In [ ]:
# girvan_newman(G) #--DOES NOT WORK COMPUTSTIONALLY HARD
newman_leading_eigenvector(G) 
newman_sl_eigenvector_community( G )
fiedler_vector_method( G )

<h5>3. Dolphins Dataset</h5>

In [ ]:
NODES = 62
G = nx.read_edgelist("datasets/dolphins.txt")

In [ ]:
# Change the mapping of the nodes: make it 0 indexing
mapping = {}
for i in G.nodes:
    mapping[str(i)] = int(i) - 1
nx.relabel_nodes(G, mapping,copy=False)
G.nodes

In [ ]:
newman_leading_eigenvector(G) # 2 is incorrectly labelled
newman_sl_eigenvector_community( G )
girvan_newman( G )
fiedler_vector_method( G )

<h5>4. Planted-Partition BenchmarkModel</h5>

In [ ]:
number_of_groups = 2
number_of_vertices = 500
p_in = 0.85
p_out = 0.15
G_ppg = nx.planted_partition_graph( l = number_of_groups , k = number_of_vertices , p_in = p_in , p_out = p_out )


<h5>5. LFR Benchmark Graph</h5>

In [ ]:
n = 100
tau1 = 2
tau2 = 3
mu = 0.8
# average_degree = 20
min_degree = 25
max_degree = 40
min_community = 50
G_LFR = nx.LFR_benchmark_graph( n = n , tau1 = tau1 , tau2 = tau2 , mu = mu
                               , min_degree = min_degree , max_degree = max_degree ,
                               min_community = min_community )
# newman_leading_eigenvector( G_LFR )
communities = {frozenset(G_LFR.nodes[v]['community']) for v in G_LFR}
print(communities)

# Normalized Mutual Information(NMI) Score

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score

# comm1 , comm2 = fiedler_vector_method( G )
# labels = generate_community_labels( comm1 , comm2 )
normalized_mutual_info_score( [0,1,2] , [2,1,0] )

In [ ]:
# comm1 , comm2 = fiedler_vector_method( G )
labels = generate_community_labels( [0,1,2] , [2,1,0] )
# normalized_mutual_info_score( ground_truth_zach , labels )